# Ler documento

Vamos usar um leitor de PDF simples. Certamente ele não vai lidar com tabelas, imagens, equações etc.

In [1]:
from PyPDF2 import PdfReader

reader = PdfReader("artigo_exemplo.pdf")
    
text = ""

for page in reader.pages:
    text += page.extract_text() + "\n"
        
text.split("\n")[100:105]

['Figure 1: Example prompts for QA (left) and attributed QA (right, following (5)).',
 '3.2 Datasets',
 'ASQA is a long-form QA dataset for factoid questions designed to evaluate a model’s performance',
 'on naturally-occurring ambiguous questions ( 23). It is made up of 948 queries and the ground truth',
 'documents are based on a 12/20/2018 Wikipedia dump with 21M passages. We use the set of five']

# Pré-processar o documento

Há muito o que ser feito para melhorar a qualidade da fonte de informação. Pode-se ver que o leitor de PDF quebra frases pela metade. Vamos pelo menos tentar juntá-las em sentenças.

In [2]:
def merge_lines(text):
    merged = []
    current_sentence = ""
    
    for line in text:
        line = line.strip()
        if not line:
            continue
            
        if current_sentence:
            line = ' ' + line
            
        current_sentence += line
        
        if ". " in current_sentence:
            parts = current_sentence.split(". ")
            for part in parts[:-1]:
                merged.append(part + ".")
            current_sentence = parts[-1]
    
    if current_sentence:
        merged.append(current_sentence)
        
    return merged

sentences = merge_lines(text.split("\n"))
sentences[0:5]

['Toward Optimal Search and Retrieval for RAG Alexandria Leto Department of Computer Science University of Colorado Boulder alex.leto@colorado.eduCecilia Aguerrebere Intel Labs cecilia.aguerrebere@intel.com Ishwar Bhati Intel Labs ishwar.s.bhati@intel.comTed Willke Intel Labs ted.willke@intel.com Mariano Tepper Intel Labs mariano.tepper@intel.comVy Ai Vo Intel Labs vy.vo@intel.com Abstract Retrieval-augmented generation (RAG) is a promising method for addressing some of the memory-related challenges associated with Large Language Models (LLMs).',
 'Two separate systems form the RAG pipeline, the retriever and the reader, and the impact of each on downstream task performance is not well-understood.',
 'Here, we work towards the goal of understanding how retrievers can be optimized for RAG pipelines for common tasks such as Question Answering (QA).',
 'We conduct experiments focused on the relationship between retrieval and RAG performance on QA and attributed QA and unveil a number of i

# Criar embeddings e armazenar no banco de dados vetorial

Vamos usar o ChromaDB e manter o banco de dados em memória.


In [3]:
import chromadb
from tqdm import tqdm

chroma_client = chromadb.Client()

collection = chroma_client.create_collection(name="artigo_exemplo")

for idx, sentence in tqdm(enumerate(sentences), total=len(sentences)):
    collection.add(
        documents=[sentence],
        ids=[f"sentence_{idx}"]
    )


100%|██████████| 404/404 [00:23<00:00, 17.26it/s]


In [4]:
# Teste simples de recuperação

# Essa query só é usada nesta célula, uma nova query para o pipeline completo será definida mais para frente
query = "What is RAG?"

results = collection.query(
    query_texts=[query],
    n_results=3, 
)

print(f"Query: {query}\n")
for idx, (doc, distance) in enumerate(zip(results['documents'][0], results['distances'][0])):
    print(f"Document: {doc}")
    print(f"Distance: {distance}\n")


Query: What is RAG?

Document: The power of noise: Redefining retrieval for RAG systems.
Distance: 0.7560287714004517

Document: RAG systems trained end-to-end (e.g.
Distance: 0.7731610536575317

Document: RAG pipelines are made up of two disparate components: a retriever, which identifies documents relevant to a query from a given corpus, and a reader, which is typically an LLM prompted with a query, the text of the retrieved documents, and instructions to use this context to generate its response.
Distance: 0.9335125088691711



# Pipeline de geração

Vamos usar a API da OpenAI, mas pode ser trocada por Gemini ou outra gratuita.

In [90]:
# A query a ser usada daqui para frente

query = "Que modelos de LLMs são avaliados e qual é o principal resultado do artigo?"

In [91]:
# Ler os templates para os prompts
# Notar que os prompts estão em português, o que não é ideal dado que o artigo sendo processado é em inglês

import yaml

with open("prompt_template.yml", "r") as file:
    prompts = yaml.safe_load(file)

system_prompt = prompts["System_Prompt"]
prompt_template = prompts["Prompt"]
prompt_template


'Usando exclusivamente as informações contidas na seção Base de Dados, responda a pergunta contida na seção Pergunta do Usuário. Se não há informações suficientes para responder a pergunta, responda com algo indicando que não há informações suficientes. Responda na mesma língua da pergunta. Não mencione a Base de Dados na resposta.\n\nAo responder, siga estas instruções:\n- Seja objetivo e direto.\n- Seja formal e profissional.\n- Se a pergunta não tiver relação com o artigo, não responda.\n\n# Pergunta do Usuário\n{query}\n\n# Base de Dados\n{chunks}\n'

In [92]:
from openai import OpenAI
import dotenv
import os

# Carregar variáveis de ambiente (devem estar no arquivo .env)
dotenv.load_dotenv(override=True)

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def get_completion(prompt, system_prompt, model="gpt-4o-mini", json_format=False):
    """
    Obtém uma resposta do modelo de linguagem usando a API OpenAI.

    Args:
        prompt (str): O prompt principal a ser enviado ao modelo
        system_prompt (str): O prompt de sistema que define o comportamento do modelo
        model (str, optional): Nome do modelo a ser usado. Padrão é "gpt-4o-mini"
        json_format (bool, optional): Se True, força a resposta em formato JSON. Padrão é False

    Returns:
        str: A resposta gerada pelo modelo
    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.0,
        max_tokens=500,
        response_format={"type": "json_object"} if json_format else None
    )
    
    return response.choices[0].message.content




## Expansão de query

Vamos processar a query para facilitar a tarefa de recuperação. Aqui fazemos duas coisas:
1. Identificamos se a query é composta (i.e. se possui mais de uma pergunta)
2. Extraímos termos relevantes para cada pergunta identificada
3. Geramos uma lista de respostas fictícias para cada pergunta identificada


In [93]:
import json


prompt = prompts["Prompt_Expansao"].format(query=query)

response = get_completion(prompt, "", json_format=True)

response_json = json.loads(response)
queries = response_json['termos']
respostas = response_json['respostas_ficticias']
print(queries)
print(respostas)


['LLM, model, models', 'results, outcomes, findings, contributions']
['The article evaluates several LLM models, including BERT, GPT-3, and T5, highlighting their performance on various benchmarks.', 'The main result of the article indicates that the proposed model outperforms existing LLMs by achieving a 15% increase in accuracy on the test dataset.']


In [94]:
# Vamos usar como queries tanto os termos como as respostas fictícias

queries = queries + respostas

In [95]:
# Recuperação e construção do prompt
all_docs = []
for query_ in queries:
    docs = collection.query(
        query_texts=[query_],
        n_results=10
    )
    all_docs.extend(docs['documents'][0])


formatted_chunks = "\n".join([f"{chunk}\n" for chunk in all_docs])

prompt = prompt_template.format(chunks=formatted_chunks, query=query)

print(prompt)

Usando exclusivamente as informações contidas na seção Base de Dados, responda a pergunta contida na seção Pergunta do Usuário. Se não há informações suficientes para responder a pergunta, responda com algo indicando que não há informações suficientes. Responda na mesma língua da pergunta. Não mencione a Base de Dados na resposta.

Ao responder, siga estas instruções:
- Seja objetivo e direto.
- Seja formal e profissional.
- Se a pergunta não tiver relação com o artigo, não responda.

# Pergunta do Usuário
Que modelos de LLMs são avaliados e qual é o principal resultado do artigo?

# Base de Dados
3 Experiment setup We conduct our experiments with two instruction-tuned LLMs: LLaMA (Llama-2-7b-chat) ( 18) and Mistral (Mistral-7B-Instruct-v0.3) ( 19).

This difference between LLMs is likely due to LLaMA’s shorter context window.

We ran LLM inference on NVIDIA GPUs of varying models (NVIDIA Titan Xp or X Pascal series, or NVIDIA A40).

Llama: Open and efficient foundation language models

In [96]:
# Geração da resposta

response = get_completion(prompt, system_prompt)
print(query)
print(response)

Que modelos de LLMs são avaliados e qual é o principal resultado do artigo?
Os modelos de LLMs avaliados no artigo são LLaMA (Llama-2-7b-chat) e Mistral (Mistral-7B-Instruct-v0.3). O principal resultado do artigo é que a performance de QA e as métricas de citação variam com o número de documentos recuperados, revelando insights úteis para o desenvolvimento de pipelines de RAG de alto desempenho.
